# Clasificación de Frutas: Entrenamiento con ResNet18

Este notebook documenta paso a paso el proceso de descarga de un dataset de frutas desde Kaggle, preprocesamiento, definición del modelo (ResNet18 con aprendizaje por transferencia), entrenamiento, evaluación y guardado del modelo entrenado.

## Paso 1: Importación de Librerías

In [ ]:
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from google.colab import drive

## Paso 2: Descarga del Dataset desde Kaggle Hub

In [ ]:
path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")
print("Path to dataset files:", path)

## Paso 3: Configuración Inicial y Transformaciones

In [ ]:
batch_size = 8
img_size = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = path

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## Paso 4: Carga y División del Dataset

In [ ]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes

val_split = 0.2
val_size = int(len(full_dataset) * val_split)
train_size = len(full_dataset) - val_size
torch.manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

## Paso 5: Definición del Modelo

In [ ]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))

model = model.to(device)

## Paso 6: Configuración del Entrenamiento

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
epochs = 10

## Paso 7: Entrenamiento del Modelo

In [ ]:
train_losses = []
val_losses = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    accuracy = 100 * correct / total
    print(f"✅ Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Val Loss: {val_loss:.4f} - Accuracy: {accuracy:.2f}%")

## Paso 8: Visualización de la Curva de Pérdida

In [ ]:
plt.plot(train_losses, label='Entrenamiento')
plt.plot(val_losses, label='Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de Pérdida')
plt.legend()
plt.show()

## Paso 9: Guardado del Modelo

In [ ]:
model_path = "fruitscan_model.pth"
torch.save(model.state_dict(), model_path)
print(f"\n✅ Modelo guardado como: {model_path}")